In [ ]:
#table joiner
def TableJoiner(filepaths):
    """
    Gets the YKR grid and merges the grid with accessibility data from chosen grid cells
    """
    i=0

    #fpgrid = "/Users/sonjakoivisto/Downloads/HelsinkiTravelTimeMatrix2018/data/MetropAccess_YKR_grid_EurefFIN.shp"
    fpgrid = "data/MetropAccess_YKR_grid_EurefFIN.shp"
    grid = gpd.read_file(fpgrid)

    #iterate over filepaths
    for fp in filepaths:

        #read in the file
        data = pd.read_csv(fp, sep=";", usecols=["from_id", "bike_f_t", "pt_r_t", "car_r_t"])
        #get the cell number and add it to all travel time columns to distinguish them
        cell_ID = fp.split("_")[-1][:-4]
        new_names = {"from_id": "YKR_ID", "pt_r_t": "pt_r_t_" + cell_ID, "bike_f_t": "bike_f_t_" + cell_ID,
        "car_r_t": "car_r_t_" + cell_ID}
        data= data.rename(columns=new_names)
    
        #merge file with grid on the id of cells and remove no data values
        grid = grid.merge(data, on="YKR_ID")
        grid.replace(to_replace=-1, value=np.nan, inplace=True)
        grid = grid.dropna()

        i+=1
    
    #if there are multiple destination points, count the minimum travel time to closest destination point
    if(len(filepaths)>1):
        for i, row in grid.iterrows(): 
    
            grid["min_t"][i] = grid.iloc[i, 4:12].min()
    
    #grid = grid.set_index("YKR_ID")
    return grid
    
geodata = TableJoiner(fps)
geodata

In [ ]:
#visualiser
def visualiser(geodata, transport_method, interactive=None): 
    """
    Takes travel times to a grid cell and visualises them on a map. First argument is a geodataframe with the travel time values.
    Second argument is a transportation method (bike, public transport or car) which you want to visualise. The third agrument
    is optional and can be added to obtain interactive map as a result.
    """
    
    #check if the minimum distance to closest destination has been counted, if so, use that
    if "min_t" in geodata.columns:
        column_name = "min_t"
        
    #make the column name which should be visualised according to user input
    if(transport_method == "bike"):
        column_name = "bike_f_t_" + cell_ID
        
    elif(transport_method == "public transport"):
        column_name = "pt_r_t_" + cell_ID
        
    elif(transport_method == "car"):
        column_name = "car_r_t_" + cell_ID
        
    else:
        print("Transport method should be one of the following: bike, public transport or car. Please insert a string")
            
    #change crs to add basemap later
    geodata = geodata.to_crs(epsg=3857)
    
    #define class breaks to array seen below (upper limits), apply this classification to pt and car travel times
    bins = [5,10,15,20,25,30,35,40,50,60]
    classifier = mapclassify.UserDefined(geodata[column_name], bins)
    
    #make new columns with class values
    geodata["classified"] = geodata[[column_name]].apply(classifier)
      
    #arrange the code somehow    
    if(type(transport_method)==list):
        transport1 = transport_method[0] + cell_ID
        transport2 = transport_method[1] + cell_ID
        geodata["difference"] = geodata[transport1] - geodata[transport2]
        

        classifier = mapclassify.UserDefined(geodata["difference"], bins)
        #make new columns with class values
        geodata["classified"] = geodata[["difference"]].apply(classifier)
        
    #produce an interactive folium map if the optional parameter is passed    
    if(interactive == "yes"):
        
        #add a basemap
        m = folium.Map(location=[60.25, 24.8], tiles = 'cartodbpositron', zoom_start=10, control_scale=True,
                       attribution = "Data: Helsinki Travel Time Matrix")

        #add the choropleth
        folium.Choropleth(
        geo_data=geodata,
        name="Travel times" + cell_ID,
        data=geodata,
        columns=["YKR_ID", column_name],
        key_on="feature.properties.YKR_ID",
        bins = 10,
        fill_color="RdYlBu",
        fill_opacity=0.7,
        line_opacity=0.2,
        line_color="white",
        line_weight=0,
        highlight=True,
        legend_name="Travel times by " + transport_method + "in minutes",
        ).add_to(m)
        
        #add tooltips (info when hovering over) as geoJson
        folium.GeoJson(geodata, name="travel time", smooth_factor=2,
              style_function=lambda x: {'weight':0.01,'color':'#807e7e', 'fillOpacity':0},
               highlight_function=lambda x: {'weight':1.5, 'color':'black'},
              tooltip=folium.GeoJsonTooltip(fields=["YKR_ID", column_name],labels=True, sticky=False)).add_to(m)
        
        #display layer control
        folium.LayerControl().add_to(m)

        #save and return the map
        outfp= "data/traveltimes" + cell_ID + ".html"
        m.save(outfp)
        
        return m
    
    #produce a static map in interactive parameter is not present
    else:
        #plot
        fig, ax = plt.subplots(figsize=(10,10))

        #plot the travel times according to the classified field
        geodata.plot(ax= ax, column="classified", cmap="RdYlBu", legend=True) 
        
        
        #add basemap
        cartodb_url = "https://a.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}.png" 

        ctx.add_basemap(ax, attribution="Source: Helsinki region travel time matrix by UH. Basemap: OSM light", 
                url=cartodb_url)


        #add scalebar
        scalebar = ScaleBar(2.0, location=4) 
        plt.gca().add_artist(scalebar)
        
        #add north arrow
        x, y, arrow_length = 0.9, 0.2, 0.115
        ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=20,
            xycoords=ax.transAxes)
        
        #add title and show map
        ax.set_title("Travel times by " + transport_method +" to "+ cell_ID, fontsize=24)
        plt.tight_layout()
        fig.show()
        
        #save and return fig
        output_fig = "outputs/traveltimes" + cell_ID + ".png"
        plt.savefig(output_fig)
        return output_fig


#visualiser(geodata, ["pt_r_t_", "bike_f_t_"])
visualiser(geodata, "bike", interactive="yes")

In [ ]:
#arrange the code somehow    
    if(type(transport_method)==list):
        transport1 = transport_method[0] + cell_ID
        transport2 = transport_method[1] + cell_ID
        geodata["difference"] = geodata[transport1] - geodata[transport2]
        

        classifier = mapclassify.UserDefined(geodata["difference"], bins)
        #make new columns with class values
        geodata["classified"] = geodata[["difference"]].apply(classifier)